In [1]:
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel

In [2]:
model_path = "/data-ai/usr/lmj/models/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_path)
processor = CLIPProcessor.from_pretrained(model_path)

In [37]:
url = "https://tydata.touying.com/album/202208/26/162550ex8g2l8lil2ahlsh.png!w800"
image = Image.open(requests.get(url, stream=True).raw)

In [45]:
type(image)

PIL.JpegImagePlugin.JpegImageFile

In [39]:
text = ["a photo of a computer", "a photo of a mouse", "a photo of a keyboard", "a photo of a cellphone"]
inputs = processor(text=text, images=image, return_tensors="pt", padding=True)

In [40]:
outputs = model(**inputs)

In [41]:
logits_per_image = outputs.logits_per_image

In [42]:
logits_per_image

tensor([[22.0141, 21.4184, 19.4461, 24.5632]], grad_fn=<TBackward0>)

In [43]:
probs = logits_per_image.softmax(dim=1)

In [44]:
probs.detach().numpy().tolist()

[[0.06933382153511047,
  0.0382152795791626,
  0.005317334085702896,
  0.8871335387229919]]

In [21]:
# image batch inference
urls = ['https://tydata.touying.com/album/202208/26/162550ex8g2l8lil2ahlsh.png!w800',
      'https://image.guojj.com/exhibition/30/18/089ee4b20a02f20c21393a5a89827f49.jpg',
      'https://openfile.meizu.com/group1/M00/0B/37/Cgbj0GTaGi6AK5TZAApI248naWQ139.png@480x480.jpg',
       'https://img2.gomein.net.cn/image//img/201204/10000107/1000151468/1000324012_360.jpg',
       'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS8GuYYzCUEs0l-Bo1PSTuLKEo2PMI3aKsdZw&usqp=CAU']
images = [Image.open(requests.get(url, stream=True).raw) for url in urls]

In [22]:
text = ["a photo of a computer", "a photo of a mouse", "a photo of a keyboard", "a photo of a mobile phone"]
inputs = processor(text=text, images=images, return_tensors="pt", padding=True)

In [23]:
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image
logits_per_image

tensor([[22.0141, 21.4184, 19.4461, 26.0394],
        [27.9471, 19.9922, 21.7536, 20.0816],
        [19.9711, 19.6626, 16.9671, 26.0194],
        [24.5486, 26.6214, 22.3131, 20.5676],
        [24.3437, 20.4304, 29.4832, 21.9115]], grad_fn=<TBackward0>)

In [24]:
probs = logits_per_image.softmax(dim=1)

In [27]:
import numpy as np
label_ids = np.argmax(probs.detach().numpy(), axis=1).tolist()

In [30]:
for label_id in label_ids:
    print(text[label_id])

a photo of a mobile phone
a photo of a computer
a photo of a mobile phone
a photo of a mouse
a photo of a keyboard
